In [1]:
import os
import uuid

from datetime import datetime, timedelta
import values, helpers
import random
import pandas as pd
import numpy as np
from create_sap_table import create_table_leanx as ctl

In [2]:
np.set_printoptions(edgeitems=30, linewidth = 500)

In [3]:
class User:
    def __init__(self, bname, ustyp, mandt=values.mandt) -> None:
        self.mandt = mandt
        self.bname = bname
        self.ustyp = ustyp

class Material:
    def __init__(self, matnr, price, availability, mandt=values.mandt) -> None:
        self.mandt = mandt
        self.matnr = matnr
        self.price = price
        self.availability = availability

class Customer:
    def __init__(self, kunnr, erdat, credit_risk, mandt=values.mandt) -> None:
        self.mandt = mandt
        self.kunnr = kunnr
        self.erdat = erdat
        self.credit_risk = credit_risk
        
class SalesOrderItem:
    def __init__(self, vbeln, posnr, mandt=values.mandt) -> None:
        self.mandt = mandt
        self.vbeln = vbeln
        self.posnr = posnr

In [4]:
cdpos_json = {}
cdhdr_json = {}
vbak_json = {}
vbkd_json = {}
vbuk_json = {}
tvakt_json = {}
tvak_json = {}
tvkot_json = {}
tvtwt_json = {}
tvsbt_json = {}
t001_json = {}
tvkbt_json = {}
dd07t_json = {}

vbap_json = {}
tvagt_json = {}
tvapt_json = {}
tvro_json = {}
usr02_json = {}

In [5]:
# insert tables and create objects list
USERS = []
for k, v in values.users.items():
    usr02_json[k] = {
        "MANDT": values.mandt,
        "BNAME": k,
        "USTYP": v
    }

    USERS.append(User(bname=k, ustyp=v))

In [6]:


CUSTOMERS = []
for k, v in values.customers.items():
    # kna1_last_index = len(KNA1)
    rand_kunnr = uuid.uuid4()
    rand_erdat = helpers.generate_random_datetime(start_date=datetime(2022, 1, 1), end_date=datetime(2023, 1, 1))

    # KNA1.loc[kna1_last_index, 'MANDT'] = values.mandt
    # KNA1.loc[kna1_last_index, 'KUNNR'] = rand_kunnr
    # KNA1.loc[kna1_last_index, 'NAME1'] = k

    CUSTOMERS.append(Customer(kunnr=rand_kunnr, erdat=rand_erdat, credit_risk=v['credit_risk']))

MATERIALS = []
for k, v in values.materials.items():
    mara_last_index = len(MATERIALS)
    rand_matnr = uuid.uuid4()

    # MARA.loc[mara_last_index, 'MANDT'] = values.mandt
    # MARA.loc[mara_last_index, 'MATNR'] = rand_matnr
    # MARA.loc[mara_last_index, 'MTART'] = k

    MATERIALS.append(Material(matnr=rand_matnr, price=v['price'], availability=v['availability']))

In [7]:
def create_sales_order(vbak_args, vbkd_args, vbuk_args, tvkot_args, tvtwt_args, tvsbt_args, t001_args, tvkbt_args, dd07t_args, tvakt_args):
    vbak_json[vbak_args["VBELN"]] = {
        "MANDT": vbak_args["MANDT"],
        "VBELN": vbak_args["VBELN"],
        "ERDAT": vbak_args["ERDAT"],
        "ERZET": vbak_args["ERZET"],
        "ERNAM": vbak_args["ERNAM"],
        "OBJNR": vbak_args["OBJNR"],
        "AUART": vbak_args['AUART'],
        "BUKRS_VF": vbak_args["BUKRS_VF"] ,
        "KUNNR": vbak_args["KUNNR"],
        "VSBED": vbak_args["VSBED"],
        "VKBUR": vbak_args["VKBUR"],
        "KKBER": vbak_args["KKBER"],
        "NETWR": vbak_args["NETWR"],
        "WAERK": vbak_args["WAERK"],
        "VBTYP": vbak_args["VBTYP"],
        "VDATU": vbak_args["VDATU"],
        "BSTNK": vbak_args["BSTNK"],
        "BSTDK": vbak_args["BSTDK"],
        "VKORG": vbak_args["VKORG"],
        "VTWEG": vbak_args["VTWEG"],
    }
    vbkd_json[f'{vbak_args["VBELN"]}_{"000000"}'] = {
        "MANDT": vbak_args["MANDT"],
        "VBELN": vbak_args["VBELN"],
        "POSNR": '000000',
        "INCO2": vbkd_args["INCO2"],
        "INCO1": vbkd_args["INCO1"],
        "ZTERM": vbkd_args["ZTERM"],
    }
    vbuk_json[vbak_args["VBELN"]] = {
        "MANDT": vbak_args["MANDT"],
        "VBELN": vbak_args["VBELN"],
        "GBSTK": vbuk_args["GBSTK"],
        "BESTK": vbuk_args["BESTK"],
    }
    tvakt_json[f'{"EN"}_{vbak_args["AUART"]}'] = {
        "MANDT": vbak_args["MANDT"],
        "SPRAS": 'EN',
        "AUART": vbak_args["AUART"],
        "BEZEI": tvakt_args["BEZEI"],
    }
    tvak_json[f'{vbak_args["MANDT"]}_{vbak_args["AUART"]}'] = {
        "MANDT": vbak_args["MANDT"],
        "AUART": vbak_args["AUART"],
        "KLIMP": "KLIMP",
    }
    tvkot_json[f'{"EN"}_{vbak_args["VKORG"]}'] = {
        "MANDT": vbak_args["MANDT"],
        "SPRAS": 'EN',
        "VKORG": vbak_args["VKORG"],
        "VTEXT": tvkot_args["VTEXT"],
    }
    tvtwt_json[f'{"EN"}_{vbak_args["VTWEG"]}'] = {
        "MANDT" : vbak_args["MANDT"],
        "SPRAS" : "EN",
        "VTWEG" : vbak_args["VTWEG"],
        "VTEXT" : tvtwt_args["VTEXT"],
    }
    tvsbt_json[f'{"EN"}_{vbak_args["VSBED"]}'] = {
        "MANDT": vbak_args["MANDT"],
        "SPRAS": "EN",
        "VSBED": vbak_args["VSBED"],
        "VTEXT": tvsbt_args["VTEXT"],
    }
    t001_json[f'{vbak_args["BUKRS_VF"]}'] = {
        "MANDT": vbak_args["MANDT"],
        "BUKRS": vbak_args["BUKRS_VF"],
        "BUTXT": t001_args["BUTXT"],
    }
    tvkbt_json[f'{"EN"}_{vbak_args["VKBUR"]}'] = {
        "MANDT": vbak_args["MANDT"],
        "SPRAS": "EN",
        "VKBUR": vbak_args["VKBUR"],
        "BEZEI": tvkbt_args["BEZEI"],
    }
    dd07t_json[f'{"KLIMP"}_{"EN"}'] = {
        "DOMNAME": "VBTYP",
        "DDLANGUAGE": "EN",
        "DOMVALUE_L": vbak_args["VBTYP"],
        "AS4LOCAL": dd07t_args["AS4LOCAL"],
        "VALPOS": dd07t_args["VALPOS"],
        "AS4VERS": dd07t_args["AS4VERS"],
        "DDTEXT": dd07t_args["DDTEXT"],
    }

def create_sales_order_item(vbak_args, vbap_args, tvagt_args, tvapt_args, tvro_args):
    vbap_json[f'{vbak_args["VBELN"]}_{vbap_args["POSNR"]}'] = { 
        "MANDT": vbak_args["MANDT"],
        "VBELN": vbak_args["VBELN"],
        "ERDAT": vbak_args["ERDAT"],
        "ERZET": vbak_args["ERZET"],
        "VGTYP": vbap_args["VGTYP"],
        "VGBEL": vbap_args["VGBEL"],
        "VGPOS": vbap_args["VGPOS"],
        "ERNAM": vbak_args["ERNAM"],
        "NETWR": vbap_args["NETWR"],
        "WAERK": vbap_args["WAERK"],
        "NETPR": vbap_args["NETPR"],
        "KPEIN": vbap_args["KPEIN"],
        "MATNR": vbap_args["MATNR"],
        "WERKS": vbap_args["WERKS"],
        "ABGRU": vbap_args["ABGRU"],
        "OBJNR": vbap_args["OBJNR"],
        "FKREL": vbap_args["FKREL"],
        "PSTYV": vbap_args["PSTYV"],
        "POSNR": vbap_args["POSNR"],
        "BRGEW": vbap_args["BRGEW"],
        "GEWEI": vbap_args["GEWEI"],
        "NTGEW": vbap_args["NTGEW"],
        "VRKME": vbap_args["VRKME"],
        "KWMENG": vbap_args["KWMENG"],
        "KDMAT": vbap_args["KDMAT"],
        "ROUTE": vbap_args["ROUTE"],
    }

    tvagt_json[f'{"EN"}_{vbap_args["ABGRU"]}'] = {
        "MANDT": vbak_args["MANDT"],
        "BEZEI": tvagt_args["BEZEI"],
        "SPRAS": "EN",
        "ABGRU": vbap_args["ABGRU"],
    }
    tvapt_json[f'{"EN"}_{vbap_args["PSTYV"]}'] = {    
        "MANDT": vbak_args["MANDT"],
        "VTEXT": tvapt_args["VTEXT"],
        "SPRAS": "EN",
        "PSTYV": vbap_args["PSTYV"],
    }
    tvro_json[f'{vbap_args["ROUTE"]}'] = {
        "MANDT": vbak_args["MANDT"],
        "TRAZTD": tvro_args["TRAZTD"],
        "ROUTE": vbap_args["ROUTE"]
    }

In [8]:
# create sales order
for i in range(100):
    vbak_vbeln = uuid.uuid4()
    customer = random.choice(CUSTOMERS)
    vbak_erdat = helpers.generate_random_datetime(start_date=customer.erdat, end_date=datetime(2024, 1, 1))
    requestd_delivery_time = vbak_erdat + helpers.UPTO_MONTH()
    latest_time = vbak_erdat
    sales_org_key = random.choice(list(values.sales_orgs.keys()))
    sales_office = random.choice(values.sales_orgs[sales_org_key]['sales_offices'])
    
    vbak_args = {
        "MANDT": values.mandt,
        "VBELN": vbak_vbeln,
        "ERDAT": vbak_erdat,
        "ERZET": helpers.generate_random_time(),
        "ERNAM": random.choice(USERS).bname,
        "OBJNR": vbak_vbeln,
        'AUART': random.choice(values.sales_order_doc_types),
        "BUKRS_VF": customer.kunnr, # we are considering each customer to be a legal entity (to simplify)
        "KUNNR": customer.kunnr,
        "VSBED": random.choice(values.shipping_conditions),
        "VKORG": sales_org_key,
        "VKBUR": sales_office,
        "KKBER": random.choice(values.credit_control_areas),
        "NETWR": 0, # FIXME add this value from VBAP
        "WAERK": 'EUR',
        "VBTYP": random.choice(values.sales_order_doc_types),
        "VDATU": requestd_delivery_time,
        "BSTNK": uuid.uuid4(),
        "BSTDK": vbak_erdat,
        "VTWEG": random.choice(values.distribution_channels)
    }

    # SalesOrderItem (VBAP)
    total_netwr = 0
    num_sales_order_item = 10
    materials = random.sample(MATERIALS, num_sales_order_item)
    for m in materials:
        quantity = random.randint(15, 100)
        total_price = m.price * 12 * 12 * quantity
        total_netwr += total_price
        vbap_args = {
            "VGTYP": 'C',
            "VGBEL": 'DEF', #FIXME
            "VGPOS": 99, #FIXME
            "NETWR": total_price,
            "WAERK": 'EUR',
            "NETPR": m.price,
            "KPEIN": 1, # FIXME
            "MATNR": m.matnr,
            "WERKS": 'DEF', #FIXME
            "ABGRU": 'DE', #FIXME
            "OBJNR": 'DEF', #FIXME
            "FKREL": None, #FIXME
            "PSTYV": 'DEF', #FIXME
            "POSNR": MATERIALS.index(m),
            "BRGEW": 99, #FIXME
            "GEWEI": 'KG',
            "NTGEW": 99, #FIXME
            "VRKME": 99, #FIXME
            "ROUTE": 'DEF', #FIXME
            "KWMENG": quantity,
            "KDMAT": m.matnr
        }
        tvagt_args = {
            "BEZEI": 'DEF', #FIXME
        }
        tvapt_args = {
            "VTEXT": 'DEF', #FIXME
        }
        tvro_args = {
            "TRAZTD": 1, #FIXME
        }
        
        create_sales_order_item(vbak_args=vbak_args, vbap_args=vbap_args, tvagt_args=tvagt_args, tvapt_args=tvapt_args, tvro_args=tvro_args)
    
    # have SO aftyer SO items creation inorder to have total_netwr passed to SO
    # SalesOrder (VBAK)
    vbak_args["NETWR"] = total_netwr
    vbkd_args = {
        "INCO1": random.choice(values.inco1_terms),
        "INCO2": random.choice(values.inco2_terms),
        "ZTERM": random.choice(values.zterms)
    }
    vbuk_args = {
        "GBSTK": None, #FIXME add overall processing status
        "BESTK": None #FIXME add confirmation status
    }
    tvakt_args = {
        "BEZEI": 'DEF' #FIXME
    }
    tvkot_args = {
        "VTEXT": 'DEF' #FIXME
    }
    tvtwt_args = {
        "VTEXT" : 'DEF' #FIXME
    }
    tvsbt_args = {
        "VTEXT" : 'DEF' #FIXME
    }
    t001_args = {
        "BUTXT": 'DEF' #FIXME
    }
    tvkbt_args = {
        "BEZEI": 'DEF' #FIXME
    }
    dd07t_args = {
        "AS4LOCAL": 'DEF', #FIXME
        "VALPOS": 99, #FIXME
        "AS4VERS": 99, #FIXME
        "DDTEXT": 'DEF', #FIXME
    }
    
    create_sales_order(vbak_args=vbak_args, vbkd_args=vbkd_args, vbuk_args=vbuk_args, tvkot_args=tvkot_args, tvtwt_args=tvtwt_args, tvsbt_args=tvsbt_args, t001_args=t001_args, tvkbt_args=tvkbt_args, dd07t_args=dd07t_args, tvakt_args=tvakt_args)
    

In [11]:
cdpos_full_col = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table('cdpos')])
cdpos = pd.concat([cdpos_full_col, pd.DataFrame(cdpos_json.values())])
cdhdr_full_col = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table('cdhdr')])
cdhdr = pd.concat([cdhdr_full_col, pd.DataFrame(cdhdr_json.values())])
vbak_full_col = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table('vbak')])
vbak = pd.concat([vbak_full_col, pd.DataFrame(vbak_json.values())])
vbkd_full_col = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table('vbkd')])
vbkd = pd.concat([vbkd_full_col, pd.DataFrame(vbkd_json.values())])
vbuk_full_col = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table('vbuk')])
vbuk = pd.concat([vbuk_full_col, pd.DataFrame(vbuk_json.values())])
tvakt_full_col = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table('tvakt')])
tvakt = pd.concat([tvakt_full_col, pd.DataFrame(tvakt_json.values())])
tvak_full_col = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table('tvak')])
tvak = pd.concat([tvak_full_col, pd.DataFrame(tvak_json.values())])
tvkot_full_col = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table('tvkot')])
tvkot = pd.concat([tvkot_full_col, pd.DataFrame(tvkot_json.values())])
tvtwt_full_col = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table('tvtwt')])
tvtwt = pd.concat([tvtwt_full_col, pd.DataFrame(tvtwt_json.values())])
tvsbt_full_col = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table('tvsbt')])
tvsbt = pd.concat([tvsbt_full_col, pd.DataFrame(tvsbt_json.values())])
t001_full_col = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table('t001')])
t001 = pd.concat([t001_full_col, pd.DataFrame(t001_json.values())])
tvkbt_full_col = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table('tvkbt')])
tvkbt = pd.concat([tvkbt_full_col, pd.DataFrame(tvkbt_json.values())])
dd07t_full_col = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table('dd07t')])
dd07t = pd.concat([dd07t_full_col, pd.DataFrame(dd07t_json.values())])

vbap_full_col = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table('vbap')])
vbap = pd.concat([vbap_full_col, pd.DataFrame(vbap_json.values())])
tvagt_full_col = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table('tvagt')])
tvagt = pd.concat([tvagt_full_col, pd.DataFrame(tvagt_json.values())])
tvapt_full_col = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table('tvapt')])
tvapt = pd.concat([tvapt_full_col, pd.DataFrame(tvapt_json.values())])
tvro_full_col = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table('tvro')])
tvro = pd.concat([tvro_full_col, pd.DataFrame(tvro_json.values())])

usr02_full_col = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table('usr02')])
usr02 = pd.concat([usr02_full_col, pd.DataFrame(usr02_json.values())])


In [14]:
cdpos.to_csv('data/ocpm/cdpos.csv', index=False)
cdhdr.to_csv('data/ocpm/cdhdr.csv', index=False)
vbak.to_csv('data/ocpm/vbak.csv', index=False)
vbkd.to_csv('data/ocpm/vbkd.csv', index=False)
vbuk.to_csv('data/ocpm/vbuk.csv', index=False)
tvakt.to_csv('data/ocpm/tvakt.csv', index=False)
tvak.to_csv('data/ocpm/tvak.csv', index=False)
tvkot.to_csv('data/ocpm/tvkot.csv', index=False)
tvtwt.to_csv('data/ocpm/tvtwt.csv', index=False)
tvsbt.to_csv('data/ocpm/tvsbt.csv', index=False)
t001.to_csv('data/ocpm/t001.csv', index=False)
tvkbt.to_csv('data/ocpm/tvkbt.csv', index=False)
dd07t.to_csv('data/ocpm/dd07t.csv', index=False)
vbap.to_csv('data/ocpm/vbap.csv', index=False)
tvagt.to_csv('data/ocpm/tvagt.csv', index=False)
tvapt.to_csv('data/ocpm/tvapt.csv', index=False)
tvro.to_csv('data/ocpm/tvro.csv', index=False)
usr02.to_csv('data/ocpm/usr02.csv', index=False)

In [43]:
old_code = { # def activity_create_sales_order_item(self, materials: list):
    #     for i, mat in enumerate(materials):
    #         quantity = random.randint(100, 250)
    #         vbap_last_index = len(VBAP)

    #         VBAP.loc[vbap_last_index, 'MANDT'] = self.mandt
    #         VBAP.loc[vbap_last_index, 'VBELN'] = self.vbeln
    #         VBAP.loc[vbap_last_index, 'POSNR'] = f'{i}'
    #         VBAP.loc[vbap_last_index, 'MATNR'] = mat.matnr
    #         VBAP.loc[vbap_last_index, 'KWMENG'] = quantity # Cumulative Order Quantity in Sales Units
    #         VBAP.loc[vbap_last_index, 'NETWR'] = mat.price * 12 * 12 * quantity # calculated as a box which usually contains 12 dozens
    #         VBAP.loc[vbap_last_index, 'VBELN'] = self.vbeln

    #         self.netwr += mat.price * 12 * 12 * quantity
    #         self.vbaps.append(SalesOrderItem(vbeln=self.vbeln, posnr=i))

    # def activity_generate_delivery_doc(self, delivery_doc_created_at: datetime, activity_by: User):        
    #     # generate the delivery document
    #     likp_last_index = len(LIKP)
    #     rand_likp_id = uuid.uuid4() # HACK for rework caes we are recreating new doc
    #     LIKP.loc[likp_last_index, 'MANDT'] = self.mandt
    #     LIKP.loc[likp_last_index, 'VBELN'] = rand_likp_id
    #     LIKP.loc[likp_last_index, 'ERDAT'] = delivery_doc_created_at
    #     LIKP.loc[likp_last_index, 'ERNAM'] = activity_by

    #     # record LIPS
    #     for vbap in self.vbaps:
    #         lips_last_index = len(LIPS)

    #         LIPS.loc[lips_last_index, 'MANDT'] = self.mandt
    #         LIPS.loc[lips_last_index, 'VBELN'] = rand_likp_id
    #         LIPS.loc[lips_last_index, 'POSNR'] = vbap.posnr
    #         LIPS.loc[lips_last_index, 'KDAUF'] = self.vbeln
    #         LIPS.loc[lips_last_index, 'KDPOS'] = vbap.posnr

    # def activity_release_delivery(self, delivery_released_at: datetime, activity_by: User):
    #     # https://leanx.eu/en/sap/table/lips.html
    #     rand_change_nr = uuid.uuid4()
        
    #     # TODO include VBLB table for release type in conneciton with VBAK, VBAP, VBUK, VBUP
    #     # record LIPS
    #     for lips_index in LIPS[LIPS['KDAUF'] == self.vbeln].index.values:
    #         value_old = LIPS.loc[lips_index, 'ABART']
    #         LIPS.loc[lips_index, 'ABART'] = 6

    #         # record change CDPOS
    #         cdpos_json.append({
    #             'MANDANT': self.mandt,
    #             'OBJECTCLAS': "LIPS",
    #             'OBJECTID': f"{self.mandt}{LIPS.loc[lips_index, 'VBELN']}{LIPS.loc[lips_index, 'POSNR']}",
    #             'CHANGENR': rand_change_nr,
    #             'TABNAME': "LIPS",
    #             'TABKEY': f"{self.mandt}{LIPS.loc[lips_index, 'VBELN']}",
    #             'FNAME': 'ABART',
    #             'CHNGIND':'U',
    #             'VALUE_OLD': value_old,
    #             'VALUE_NEW': LIPS.loc[lips_index, 'ABART']
    #         })

    #     cdhdr_json.append({
    #         'MANDANT': self.mandt,
    #         'OBJECTCLAS': "LIPS",
    #         'OBJECTID': LIPS.loc[lips_index, 'KDAUF'],
    #         'CHANGENR': rand_change_nr,
    #         'USERNAME': activity_by,
    #         'UDATE': delivery_released_at
    #     })

    # def activity_ship_goods(self, shipped_at: datetime, activity_by: User):
    #     # record new MKPF
    #     mkpf_last_index = len(MKPF)

    #     # HACK for rework caes we are recreating new doc
    #     rand_mkpf_mblnr = uuid.uuid4()
    #     MKPF.loc[mkpf_last_index, 'MANDT'] = self.mandt
    #     MKPF.loc[mkpf_last_index, 'MBLNR'] = rand_mkpf_mblnr
    #     MKPF.loc[mkpf_last_index, 'SPE_BUDAT_UHR'] = shipped_at
    #     MKPF.loc[mkpf_last_index, 'USNAM'] = activity_by

    #     # record new MSEG
    #     for vbap in self.vbaps:
    #         mseg_last_index = len(MSEG)

    #         MSEG.loc[mseg_last_index, 'MANDT'] = self.mandt
    #         MSEG.loc[mseg_last_index, 'MBLNR'] = rand_mkpf_mblnr
    #         MSEG.loc[mseg_last_index, 'ZEILE'] = vbap.posnr
    #         MSEG.loc[mseg_last_index, 'KDAUF'] = self.vbeln
    #         MSEG.loc[mseg_last_index, 'KDPOS'] = vbap.posnr
    
    # def activity_create_billing_document(self, invoice_sent_at: datetime, activity_by: User):
    #     # record new billing document (VBRK)
    #     vbrk_last_index = len(VBRK)
    #     rand_vbrk_id = uuid.uuid4() # HACK for rework caes we are recreating new doc
    #     rand_bkpf_id = uuid.uuid4() # HACK for rework caes we are recreating new doc
    #     VBRK.loc[vbrk_last_index, 'MANDT'] = self.mandt
    #     VBRK.loc[vbrk_last_index, 'VBELN'] = rand_vbrk_id
    #     VBRK.loc[vbrk_last_index, 'ERNAM'] = activity_by
    #     VBRK.loc[vbrk_last_index, 'ERDAT'] = invoice_sent_at

    #     # create VBRP
    #     for vbap in self.vbaps:
    #         vbrp_last_index = len(VBRP)
    #         VBRP.loc[vbrp_last_index, 'MANDT'] = self.mandt
    #         VBRP.loc[vbrp_last_index, 'VBELN'] = rand_vbrk_id
    #         VBRP.loc[vbrp_last_index, 'POSNR'] = vbap.posnr
    #         VBRP.loc[vbrp_last_index, 'AUBEL'] = self.vbeln
    #         VBRP.loc[vbrp_last_index, 'AUPOS'] = vbap.posnr

    #     # record new invoice (BKPF)  ---Create Invoice---
    #     bkpf_last_index = len(BKPF)
    #     BKPF.loc[bkpf_last_index, 'MANDT'] = self.mandt
    #     BKPF.loc[bkpf_last_index, 'BUKRS'] = self.customer
    #     BKPF.loc[bkpf_last_index, 'BELNR'] = rand_bkpf_id
    #     BKPF.loc[bkpf_last_index, 'BLDAT'] = invoice_sent_at
    #     BKPF.loc[bkpf_last_index, 'USNAM'] = activity_by
    #     BKPF.loc[bkpf_last_index, 'AWKEY'] = rand_vbrk_id
    #     BKPF.loc[bkpf_last_index, 'AWTYP'] = 'VBRK'

    #     # record invoice item (BSEG)
    #     for vbap in self.vbaps:
    #         bseg_last_index = len(BSEG)
    #         BSEG.loc[bseg_last_index, 'MANDT'] = self.mandt
    #         BSEG.loc[bseg_last_index, 'BUKRS'] = self.customer
    #         BSEG.loc[bseg_last_index, 'BELNR'] = rand_bkpf_id
    #         BSEG.loc[bseg_last_index, 'VBEL2'] = self.vbeln
    #         BSEG.loc[bseg_last_index, 'POSN2'] = vbap.posnr
    #         BSEG.loc[bseg_last_index, 'BUZEI'] = vbap.posnr

    # def activity_receive_delivery_confirmation(self, actual_delivery_at: datetime, delivery_confirmation_received_at: datetime, activity_by: User):
    #     # update delivery document header
    #     likp_id = LIPS[LIPS['KDAUF'] == self.vbeln]['VBELN'].values[0]
    #     likp_index = LIKP[LIKP['VBELN'] == likp_id].index.values[0]
        
    #     value_old = LIKP.loc[likp_index, 'SPE_ACC_APP_STS']
    #     LIKP.loc[likp_index, 'SPE_ACC_APP_STS'] = 'C'
    #     LIKP.loc[likp_index, 'LFUHR'] = actual_delivery_at

    #     rand_change_nr = uuid.uuid4()

    #     # record change CDPOS
    #     cdpos_json.append({
    #         'MANDANT': self.mandt,
    #         'OBJECTCLAS': "LIKP",
    #         'OBJECTID': f"{self.mandt}{likp_id}",
    #         'CHANGENR': rand_change_nr,
    #         'TABNAME': "LIKP",
    #         'TABKEY': f"{self.mandt}{likp_id}",
    #         'FNAME': 'SPE_ACC_APP_STS',
    #         'CHNGIND':'U',
    #         'VALUE_OLD': value_old,
    #         'VALUE_NEW': LIKP.loc[likp_index, 'SPE_ACC_APP_STS'],
    #     })

    #     cdhdr_json.append({
    #         'MANDANT': self.mandt,
    #         'OBJECTCLAS': "LIKP",
    #         'OBJECTID': likp_id,
    #         'CHANGENR': rand_change_nr,
    #         'USERNAME': activity_by,
    #         'UDATE': delivery_confirmation_received_at,
    #     })

    # def activity_clear_invoice(self, inovice_cleared_at: datetime):
    #     bseg_indices = BSEG[BSEG['VBEL2'] == self.vbeln].index.values
    #     for bseg_index in bseg_indices:
    #         BSEG.loc[bseg_index, 'AUGDT'] = inovice_cleared_at
    
    # # Deveiations
    # # -----------
    # def activity_set_billing_block(self, reason_for_billing_block, billing_block_set_at: datetime, activity_by: User):
    #     vbak_indices = VBAK[VBAK['VBELN'] == self.vbeln].index.values # FIXME assert there is only one VBAK with such VBELN, currently works for all such VBAKs
        
    #     # change VBAK
    #     for vbak_index in vbak_indices:
    #         value_old = VBAK.loc[vbak_index, 'FAKSK']
    #         VBAK.loc[vbak_index, 'FAKSK'] = reason_for_billing_block # TODO currently on VBAK level, check if it needs to be in VBAP level as well
    #         rand_change_nr = uuid.uuid4()

    #         # record change CDPOS
    #         cdpos_json.append({
    #             'MANDANT': self.mandt,
    #             'OBJECTCLAS': "VBAK",
    #             'OBJECTID': f"{self.mandt}{self.vbeln}",
    #             'CHANGENR': rand_change_nr,
    #             'TABNAME': "VBAK",
    #             'TABKEY': f"{self.mandt}{self.vbeln}",
    #             'FNAME': 'FAKSK',
    #             'CHNGIND':'U',
    #             'VALUE_OLD': value_old,
    #             'VALUE_NEW': VBAK.loc[vbak_index, 'FAKSK'],
    #         })

    #         # record change CDHDR
    #         cdhdr_json.append({
    #             'MANDANT': self.mandt,
    #             'OBJECTCLAS': "VBAK",
    #             'OBJECTID': self.vbeln,
    #             'CHANGENR': rand_change_nr,
    #             'USERNAME': activity_by,
    #             'UDATE': billing_block_set_at
    #         })

    # def activity_remove_billing_block(self, billing_block_removed_at: datetime, activity_by: User):
    #     vbak_indices = VBAK[VBAK['VBELN'] == self.vbeln].index.values # FIXME assert there is only one VBAK with such VBELN, currently works for all such VBAKs

    #     # change VBAK
    #     for vbak_index in vbak_indices:
    #         value_old = VBAK.loc[vbak_index, 'FAKSK']
    #         VBAK.loc[vbak_index, 'FAKSK'] = np.nan # TODO currently on VBAK level, check if it needs to be in VBAP level as well
    #         rand_change_nr = uuid.uuid4()

    #         # record change CDPOS
    #         cdpos_json.append({
    #             'MANDANT': self.mandt,
    #             'OBJECTCLAS': "VBAK",
    #             'OBJECTID': f"{self.mandt}{self.vbeln}",
    #             'CHANGENR': rand_change_nr,
    #             'TABNAME': "VBAK",
    #             'TABKEY': f"{self.mandt}{self.vbeln}",
    #             'FNAME': 'FAKSK',
    #             'CHNGIND':'U',
    #             'VALUE_OLD': value_old,
    #             'VALUE_NEW': VBAK.loc[vbak_index, 'FAKSK'],
    #         })

    #         # record change CDHDR
    #         cdhdr_json.append({
    #             'MANDANT': self.mandt,
    #             'OBJECTCLAS': "VBAK",
    #             'OBJECTID': self.vbeln,
    #             'CHANGENR': rand_change_nr,
    #             'USERNAME': activity_by,
    #             'UDATE': billing_block_removed_at,
    #         })

    # def activity_set_delivery_block(self, reason_for_delivery_block, delivery_block_set_at: datetime, activity_by: User):
    #     vbak_indices = VBAK[VBAK['VBELN'] == self.vbeln].index.values # FIXME assert there is only one VBAK with such VBELN, currently works for all such VBAKs

    #     # change VBAK
    #     for vbak_index in vbak_indices:
    #         value_old = VBAK.loc[vbak_index, 'LIFSK']
    #         VBAK.loc[vbak_index, 'LIFSK'] = reason_for_delivery_block # TODO currently on VBAK level, check if it needs to be in VBAP level as well
    #         rand_change_nr = uuid.uuid4()

    #         # record change CDPOS
    #         cdpos_json.append({
    #             'MANDANT': self.mandt,
    #             'OBJECTCLAS': "VBAK",
    #             'OBJECTID': f"{self.mandt}{self.vbeln}",
    #             'CHANGENR': rand_change_nr,
    #             'TABNAME': "VBAK",
    #             'TABKEY': f"{self.mandt}{self.vbeln}",
    #             'FNAME': 'LIFSK',
    #             'CHNGIND':'U',
    #             'VALUE_OLD': value_old,
    #             'VALUE_NEW': VBAK.loc[vbak_index, 'LIFSK'],
    #         })

    #         # record change CDHDR
    #         cdhdr_json.append({
    #             'MANDANT': self.mandt,
    #             'OBJECTCLAS': "VBAK",
    #             'OBJECTID': self.vbeln,
    #             'CHANGENR': rand_change_nr,
    #             'USERNAME': activity_by,
    #             'UDATE': delivery_block_set_at,
    #         })

    # def activity_remove_delivery_block(self, delivery_block_removed_at: datetime, activity_by: User):
    #     vbak_indices = VBAK[VBAK['VBELN'] == self.vbeln].index.values # FIXME assert there is only one VBAK with such VBELN, currently works for all such VBAKs

    #     # change VBAK
    #     for vbak_index in vbak_indices:
    #         value_old = VBAK.loc[vbak_index, 'LIFSK']
    #         VBAK.loc[vbak_index, 'LIFSK'] = np.nan # TODO currently on VBAK level, check if it needs to be in VBAP level as well
    #         rand_change_nr = uuid.uuid4()

    #         cdpos_json.append({
    #             'MANDANT': self.mandt,
    #             'OBJECTCLAS': "VBAK",
    #             'OBJECTID': f"{self.mandt}{self.vbeln}",
    #             'CHANGENR': rand_change_nr,
    #             'TABNAME': "VBAK",
    #             'TABKEY': f"{self.mandt}{self.vbeln}",
    #             'FNAME': 'LIFSK',
    #             'CHNGIND':'U',
    #             'VALUE_OLD': value_old,
    #             'VALUE_NEW': VBAK.loc[vbak_index, 'LIFSK'],
    #         })

    #         cdhdr_json({
    #             'MANDANT': self.mandt,
    #             'OBJECTCLAS': "VBAK",
    #             'OBJECTID': self.vbeln,
    #             'CHANGENR': rand_change_nr,
    #             'USERNAME': activity_by,
    #             'UDATE': delivery_block_removed_at
    #         })

    # def activity_return_order(self, order_returned_at: datetime, activity_by: User): # TODO add retrun by order item instead of order doc
    #     vbak_indices = VBAK[VBAK['VBELN'] == self.vbeln].index.values # FIXME assert there is only one VBAK with such VBELN, currently works for all such VBAKs

    #     # change VBAK
    #     for vbak_index in vbak_indices:
    #         value_old = VBAK.loc[vbak_index, 'VBTYP']
    #         VBAK.loc[vbak_index, 'VBTYP'] = 'H' # TODO currently on VBAK level, check if it needs to be in VBAP level as well
    #         rand_change_nr = uuid.uuid4()
            
    #         cdpos_json.append({
    #             'MANDANT': self.mandt,
    #             'OBJECTCLAS': "VBAK",
    #             'OBJECTID': f"{self.mandt}{self.vbeln}",
    #             'CHANGENR': rand_change_nr,
    #             'TABNAME': "VBAK",
    #             'TABKEY': f"{self.mandt}{self.vbeln}",
    #             'FNAME': 'VBTYP',
    #             'CHNGIND':'U',
    #             'VALUE_OLD': value_old,
    #             'VALUE_NEW': VBAK.loc[vbak_index, 'VBTYP']
    #         })

    #         cdhdr_json.append({
    #             'MANDANT': self.mandt,
    #             'OBJECTCLAS': "VBAK",
    #             'OBJECTID': self.vbeln,
    #             'CHANGENR': rand_change_nr,
    #             'USERNAME': activity_by,
    #             'UDATE': order_returned_at
    #         })

    # def activity_cancel_order(self, order_cancelled_at: datetime, activity_by: User):
    #     vbak_indices = VBAK[VBAK['VBELN'] == self.vbeln].index.values # FIXME assert there is only one VBAK with such VBELN, currently works for all such VBAKs

    #     # change VBAK
    #     for vbak_index in vbak_indices:
    #         value_old = VBAK.loc[vbak_index, 'VBTYP']
    #         VBAK.loc[vbak_index, 'VBTYP'] = 'CANCELLED' # TODO currently on VBAK level, check if it needs to be in VBAP level as well
    #         rand_change_nr = uuid.uuid4()

    #         cdpos_json.append({
    #             'MANDANT': self.mandt,
    #             'OBJECTCLAS': "VBAK",
    #             'OBJECTID': f"{self.mandt}{self.vbeln}",
    #             'CHANGENR': rand_change_nr,
    #             'TABNAME': "VBAK",
    #             'TABKEY': f"{self.mandt}{self.vbeln}",
    #             'FNAME': 'VBTYP',
    #             'CHNGIND':'U',
    #             'VALUE_OLD': value_old,
    #             'VALUE_NEW': VBAK.loc[vbak_index, 'VBTYP']
    #         })

    #         cdhdr_json.append({
    #             'MANDANT': self.mandt,
    #             'OBJECTCLAS': "VBAK",
    #             'OBJECTID': self.vbeln,
    #             'CHANGENR': rand_change_nr,
    #             'USERNAME': activity_by,
    #             'UDATE': order_cancelled_at,
    #         })
        }

In [17]:
random.random()

0.9638451251481035

In [57]:
def get_user(automation_prob: float):
    return random.random() > automation_prob

In [58]:
for i in range(10):
    transition_matrix = np.array([
   # TO gdvd   rldv   shgd   sinv   rdvc   cinv   sblb   rblb   sdvb   rdvb   rord   cord   end.    # FROM (row)
       [0.001, 1.000, 0.010, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000], # gdvd
       [0.000, 0.001, 1.000, 0.010, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000], # rldv
       [0.000, 0.000, 0.001, 1.000, 0.010, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000], # shgd
       [0.000, 0.000, 0.000, 0.001, 1.000, 0.010, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000], # sinv
       [0.000, 0.000, 0.000, 0.000, 0.001, 1.000, 0.010, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000], # rdvc
       [0.000, 0.000, 0.000, 0.000, 0.000, 0.001, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 1.000], # cinv

       [0.000, 0.000, 0.950, 0.000, 0.000, 0.000, 0.000, 0.050, 0.000, 0.000, 0.000, 0.000, 0.000], # sblb
       [0.000, 0.000, 0.000, 0.950, 0.000, 0.000, 0.050, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000], # rblb
       [0.000, 0.050, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.950, 0.000, 0.000, 0.000], # sdvb
       [0.000, 0.950, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.050, 0.000, 0.000, 0.000, 0.000], # rdvb
       [0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 1.000], # rord
       [0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 1.000], # cord

       [0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 1.000], # end.
    ])
    
    automation_prob = random.uniform(0, 1)
    if automation_prob > 0.9:
        activity_by = 'BATCH_JOB'
    else:
        activity_by = random.choice(USERS).bname
        
    customer = random.choice(CUSTOMERS)
    sales_order_created_at = helpers.generate_random_datetime(start_date=customer.erdat, end_date=datetime(2024, 1, 1))
    latest_time = sales_order_created_at

    shipping_condition = 'Standard'
    materials = random.sample(MATERIALS, k=random.randint(15, 35)) # sample without replacement
    agreed_delivery_time = latest_time + helpers.UPTO_WEEK()

    sales_order = SalesOrder(ernam=activity_by, erdat=latest_time, vbtyp=shipping_condition, customer=customer.kunnr, agreed_delivery_time=agreed_delivery_time)
    sales_order.activity_create_sales_order_item(materials=materials)

    # update latest time to reflect time it took to generate delivery doc
    if activity_by == 'BATCH_JOB':
        latest_time += helpers.UPTO_DAY()
    else:
        latest_time += helpers.UPTO_DAY() + helpers.UPTO_DAY()


    activities = {
        'generate_delivery_doc': sales_order.activity_generate_delivery_doc,
        'release_delivery': sales_order.activity_release_delivery,
        'ship_goods': sales_order.activity_ship_goods,
        'create_billing_document': sales_order.activity_create_billing_document,
        'receive_delivery_confirmation': sales_order.activity_receive_delivery_confirmation,
        'clear_invoice': sales_order.activity_clear_invoice,

        'set_billing_block': sales_order.activity_set_billing_block,
        'remove_billing_block': sales_order.activity_remove_billing_block,
        'set_delivery_block': sales_order.activity_set_delivery_block,
        'remove_delivery_block': sales_order.activity_remove_delivery_block,
        'return_order': sales_order.activity_return_order,
        'cancel_order': sales_order.activity_cancel_order,

        'end' : 'end'
    }

    a_i = [
        'generate_delivery_doc',
        'release_delivery',
        'ship_goods',
        'create_billing_document',
        'receive_delivery_confirmation',
        'clear_invoice',

        'set_billing_block',
        'remove_billing_block',
        'set_delivery_block',
        'remove_delivery_block',
        'return_order',
        'cancel_order',

        'end' ,
    ]

    step = 0
    old_transition_matrix = transition_matrix.copy() # BUG previous value tracking
    steps = []
    while step < len(activities)-1:
        if step == 0: # generate delivery document
            delivery_doc_generated_by = 'BATCH_JOB' if get_user(0.5) else random.choice(USERS).bname

            # take a step
            activities['generate_delivery_doc'](delivery_doc_created_at=latest_time, activity_by=delivery_doc_generated_by)

            # NOTE Effect of the activity 
            # time
            avg_material_availability = sum([a.availability for a in materials]) / len(materials)
            if delivery_doc_generated_by == 'BATCH_JOB':
                latest_time += timedelta(hours=int(1/avg_material_availability))
            else:
                latest_time += timedelta(hours=int(2/avg_material_availability)) # manual work takes twice the amount of time

            # delivery block
            if avg_material_availability < 0.75:
                transition_matrix[a_i.index('generate_delivery_doc'), a_i.index('set_delivery_block')] = 0.5 # 30% likely to set delivery block

        elif step == 1: # release delivery
            delivery_released_by = 'BATCH_JOB' if get_user(0.9) else random.choice(USERS).bname
                
            # take a step
            activities['release_delivery'](delivery_released_at=latest_time, activity_by=delivery_released_by)

            # NOTE Effect by the activity 
            # time
            if delivery_released_by == 'BATCH_JOB':
                latest_time += helpers.UPTO_DAY()
            else:
                latest_time += helpers.UPTO_DAY() + helpers.UPTO_DAY()

            # billing block
            if customer.credit_risk > 0.6:
                transition_matrix[a_i.index('release_delivery'), a_i.index('set_billing_block')] = 0.5 # 30% likely to set billing block

        elif step == 2: # ship goods
            goods_issue_by = 'BATCH_JOB'if get_user(0.5) else random.choice(USERS).bname

            # take a step
            activities['ship_goods'](shipped_at=latest_time, activity_by=goods_issue_by)

            # NOTE Effect by the activity 
            # time
            if delivery_released_by == 'BATCH_JOB':
                latest_time += helpers.UPTO_DAY()
            else:
                latest_time += helpers.UPTO_DAY() + helpers.UPTO_DAY()

            # cancel order
            if (latest_time - agreed_delivery_time) > timedelta(0): # if the cycle_time > agreed_delivery_time
                # increase probability as it approaches 7 days
                transition_matrix[a_i.index('ship_goods'), a_i.index('cancel_order')] = 0.3
            
        elif step == 3: # send invoice
            invoice_sent_by = 'BATCH_JOB' if get_user(0.5) else random.choice(USERS).bname

            # take a step
            activities['create_billing_document'](invoice_sent_at=latest_time, activity_by=invoice_sent_by)

            # NOTE Effect by the activity 
            # time
            if delivery_released_by == 'BATCH_JOB':
                latest_time += helpers.UPTO_WEEK()
            else:
                latest_time += helpers.UPTO_WEEK() + helpers.UPTO_WEEK()

            # cancel order
            if (latest_time - agreed_delivery_time) > timedelta(0): # if the cycle_time > agreed_delivery_time
                # increase probability as it approaches 7 days
                transition_matrix[a_i.index('create_billing_document'), a_i.index('cancel_order')] = 0.3

        elif step == 4: # receive delivery confirmation
            confirmation_received_by = 'BATCH_JOB' if get_user(0.5) else random.choice(USERS).bname

            # take a step
            activities['receive_delivery_confirmation'](actual_delivery_at=latest_time, delivery_confirmation_received_at=latest_time, activity_by=confirmation_received_by)

            # NOTE Effect by the activity 
            # time
            latest_time += timedelta(days=int(10/customer.credit_risk)) # DSO based on credit_risk

            # reuturn order
            if (latest_time - agreed_delivery_time) > timedelta(0): # if the cycle_time > agreed_delivery_time
                # increase probability as it approaches 15 days
                transition_matrix[a_i.index('receive_delivery_confirmation'), a_i.index('return_order')] = 0.3

        elif step == 5: # clear invoice
            # take a step
            activities['clear_invoice'](inovice_cleared_at=latest_time)

        # Deviations
        elif step == 6: # set billing block
            billing_block_set_by = 'BATCH_JOB' if get_user(0.5) else random.choice(USERS).bname

            # take a step
            activities['set_billing_block'](reason_for_billing_block='Credit risk', billing_block_set_at=latest_time, activity_by=billing_block_set_by)

            # NOTE Effect by the activity 
            # time
            if billing_block_set_by == 'BATCH_JOB':
                latest_time += helpers.UPTO_DAY()
            else:
                latest_time += helpers.UPTO_DAY() + helpers.UPTO_DAY()

            # revert to remove billing block
            transition_matrix[a_i.index('ship_goods'), a_i.index('remove_billing_block')] = 19 # 95% chance for shg -> rblb
 
        elif step == 7: # remove billing block
            billing_block_removed_by = 'BATCH_JOB' if get_user(0.5) else random.choice(USERS).bname

            # take a step
            activities['remove_billing_block'](billing_block_removed_at=latest_time, activity_by=billing_block_removed_by)

            # NOTE Effect by the activity 
            # time
            if billing_block_set_by == 'BATCH_JOB':
                latest_time += helpers.UPTO_HOUR()
            else:
                latest_time += helpers.UPTO_HOUR() + helpers.UPTO_HOUR()

        elif step == 8: # set delivery block
            delivery_block_set_by = 'BATCH_JOB' if get_user(0.5) else random.choice(USERS).bname

            # take a step
            activities['set_delivery_block'](reason_for_delivery_block='Material availability', delivery_block_set_at=latest_time, activity_by=delivery_block_set_by)

            # NOTE Effect by the activity 
            # time
            if delivery_block_set_by == 'BATCH_JOB':
                latest_time += helpers.UPTO_WEEK()
            else:
                latest_time += helpers.UPTO_WEEK() + helpers.UPTO_WEEK() 
            # no need to revert to remove delivery block because the next step is 95% remove delivery block anyway

        elif step == 9: # remove delivery block
            delivery_block_removed_by = 'BATCH_JOB' if get_user(0.5) else random.choice(USERS).bname

            # take a step
            activities['remove_delivery_block'](delivery_block_removed_at=latest_time, activity_by=delivery_block_removed_by)

            # NOTE Effect by the activity 
            # time
            if delivery_block_removed_by == 'BATCH_JOB':
                latest_time += helpers.UPTO_HOUR()
            else:
                latest_time += helpers.UPTO_HOUR() + helpers.UPTO_HOUR() 

        elif step == 10: # return order
            # take a step
            activities['return_order'](order_returned_at=latest_time, activity_by=random.choice(USERS).bname) # FIXME add a return goods logic that innitiates its own path

        elif step == 11: # cancel order
            # take a step
            activities['cancel_order'](order_cancelled_at=latest_time,  activity_by=random.choice(USERS).bname) # NOTE if we cancel order we go into end, no need to update deviation probability

        # ---- debugging ----
        if not (
            len(VBRK['VBELN'].unique()) == len(VBRK['VBELN']) and
            len(BKPF['BELNR'].unique()) == len(BKPF['BELNR']) and
            len(LIKP['VBELN'].unique()) == len(LIKP['VBELN']) and
            len(VBAK['VBELN'].unique()) == len(VBAK['VBELN']) and 
            len(MKPF['MBLNR'].unique()) == len(MKPF['MBLNR'])
        ):
            raise IndexError('Duplicate index recorded')
        # ---- debugging ----

        # update transition matrix to sum to 1
        transition_matrix = transition_matrix / transition_matrix.sum(axis=1, keepdims=True)

        # select next step
        step = np.random.choice(len(activities), p=transition_matrix[step])


In [8]:
version = str(datetime.now())
os.mkdir(f'data/{version}')

VBAK.dropna(axis=1, how='all', inplace=True)
VBAK.to_csv(f'data/{version}/VBAK.csv', index=False)

VBAP.dropna(axis=1, how='all', inplace=True)
VBAP.to_csv(f'data/{version}/VBAP.csv', index=False)


LIKP.dropna(axis=1, how='all', inplace=True)
LIKP.to_csv(f'data/{version}/LIKP.csv', index=False)

LIPS.dropna(axis=1, how='all', inplace=True)
LIPS.to_csv(f'data/{version}/LIPS.csv', index=False)


MKPF.dropna(axis=1, how='all', inplace=True)
MKPF.to_csv(f'data/{version}/MKPF.csv', index=False)

MSEG.dropna(axis=1, how='all', inplace=True)
MSEG.to_csv(f'data/{version}/MSEG.csv', index=False)


BKPF.dropna(axis=1, how='all', inplace=True)
BKPF.to_csv(f'data/{version}/BKPF.csv', index=False)

BSEG.dropna(axis=1, how='all', inplace=True)
BSEG.to_csv(f'data/{version}/BSEG.csv', index=False)


VBRK.dropna(axis=1, how='all', inplace=True)
VBRK.to_csv(f'data/{version}/VBRK.csv', index=False)

VBRP.dropna(axis=1, how='all', inplace=True)
VBRP.to_csv(f'data/{version}/VBRP.csv', index=False)


CDHDR.dropna(axis=1, how='all', inplace=True)
CDHDR.to_csv(f'data/{version}/CDHDR.csv', index=False)

CDPOS.dropna(axis=1, how='all', inplace=True)
CDPOS.to_csv(f'data/{version}/CDPOS.csv', index=False)


USR02.dropna(axis=1, how='all', inplace=True)
USR02.to_csv(f'data/{version}/USR02.csv', index=False)

MARA.dropna(axis=1, how='all', inplace=True)
MARA.to_csv(f'data/{version}/MARA.csv', index=False)

KNA1.dropna(axis=1, how='all', inplace=True)
KNA1.to_csv(f'data/{version}/KNA1.csv', index=False)


In [9]:
VBAP

,MANDT,VBELN,POSNR,MATNR,NETWR,KWMENG
0,SC1,36781dd6-378d-4ed1-9355-9e24efdd5a35,0,0eb66a25-ef7f-4bd4-8f39-1fac73623e02,76736.16,191
1,SC1,36781dd6-378d-4ed1-9355-9e24efdd5a35,1,e106ddd8-c8bb-473a-ace4-b2986484bffe,112613.76,196
2,SC1,36781dd6-378d-4ed1-9355-9e24efdd5a35,2,e6e44c7c-9116-4e8a-957f-7ce19aea0a87,74011.68,103
3,SC1,36781dd6-378d-4ed1-9355-9e24efdd5a35,3,6a708337-5cb9-4b59-ada6-656526386288,65875.68,153
4,SC1,36781dd6-378d-4ed1-9355-9e24efdd5a35,4,bbf94f69-a6a0-479f-98f2-13aaa1f88625,36897.12,117
...,...,...,...,...,...,...
2573,SC1,92481554-ed1e-4f40-83c3-32bda9d2b3a9,23,e6e44c7c-9116-4e8a-957f-7ce19aea0a87,147304.8,205
2574,SC1,92481554-ed1e-4f40-83c3-32bda9d2b3a9,24,cd763f69-bcc6-41f2-ae0d-6696f033a8a1,45849.6,160
2575,SC1,92481554-ed1e-4f40-83c3-32bda9d2b3a9,25,d22a48bf-2cd8-413e-88fc-1cd46fc27fb4,98542.08,208
2576,SC1,92481554-ed1e-4f40-83c3-32bda9d2b3a9,26,6f1ff5b9-c1d4-498c-9427-0f90dc0c92d7,96014.88,223


In [56]:
df = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table('vbak')])

In [57]:
df

,MANDT,VBELN,ERDAT,ERZET,ERNAM,ANGDT,BNDDT,AUDAT,VBTYP,TRVOG,...,MOD_ALLOW,CANCEL_ALLOW,PAY_METHOD,BPN,REP_FREQ,LOGSYSB,KALCD,MULTI,SPPAYM,WTYSC_CLM_HDR


In [59]:
pd.concat([df, pd.DataFrame(vbak_json.values())])

,MANDT,VBELN,ERDAT,ERZET,ERNAM,ANGDT,BNDDT,AUDAT,VBTYP,TRVOG,...,MOD_ALLOW,CANCEL_ALLOW,PAY_METHOD,BPN,REP_FREQ,LOGSYSB,KALCD,MULTI,SPPAYM,WTYSC_CLM_HDR
0,SC1,8bb43129-f62d-4439-a0cd-2cb0bf63e262,2022-09-04 01:26:25,11:47:38,Francis Taylor,NaN,NaN,NaN,I,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,SC1,4b9c5531-5dc7-42ac-80dc-b548715a229f,2023-01-29 18:35:58,01:00:14,Ibrahim Lewis,NaN,NaN,NaN,C,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,SC1,7e1f9221-f0bd-4fa0-a0ab-a63bc4ab46e2,2023-02-23 03:41:52,07:38:29,Francis Taylor,NaN,NaN,NaN,I,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,SC1,cd9e979e-cd08-4745-bdf7-7b5562f8d3aa,2023-07-12 21:14:41,09:44:57,Taylor Turner,NaN,NaN,NaN,C,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,SC1,500d882e-eea2-44c5-bfbe-4dfb21bc95cf,2023-10-10 03:09:24,11:23:28,Taylor Turner,NaN,NaN,NaN,C,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,SC1,7eafd6b6-f59f-47f6-9f71-aba242c9d39b,2023-01-27 13:13:41,00:29:48,Catelyn Stark,NaN,NaN,NaN,I,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,SC1,ab032a61-caac-4028-a9be-5487eb6a390a,2023-03-08 03:23:12,03:29:07,Carmen Davis,NaN,NaN,NaN,C,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,SC1,3ad70236-4855-421f-887b-8e57bce635cb,2023-07-05 10:16:56,03:12:01,Jaden Hall,NaN,NaN,NaN,C,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,SC1,d3019d08-de64-4d6f-83fe-0f262bd9e65f,2022-11-23 18:33:14,08:25:09,Olive White,NaN,NaN,NaN,I,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,SC1,8e6e04ad-8657-4510-8800-e8829126c17b,2022-12-01 19:07:14,02:55:36,Peyton Turner,NaN,NaN,NaN,I,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
